# Creating a Recommender Agent with Client-Side Tools
**Image:Data Science 3.0, Kernel: Python3, Instance: ml.t3.medium 2vCPU + 4GiB**

**NOTE:** Before continuining with this Notebook, you will need to go through notebook [01_Recommender-Agent_Configure-Personalize-Resources.ipynb](./01_Recommender-Agent_Configure-Personalize-Resources.ipynb) to deploy the required Amazon Personalize resources.

In this Notebook, we'll demonstrate how to create a recommender agent using [Amazon Bedrock Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html). The Converse API makes it easy to incorporate tool use into your AI applications by enabling models to generate more useful and relevant responses, providing a consistent API for using tools across models, and allowing for more advanced conversational applications with external data and capabilities.

The chatbot will be able to look up customer information, retrieve order details, recommended products, similar items, product details and purchase products on behalf of the customer. We'll define the necessary tools: Amazon Personalize resources together with synthetic responses to showcase the agent's capabilities.

## Using Amazon Bedrock


[Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that makes base models from Amazon and third-party model providers accessible through an API.

<div class="alert alert-block alert-warning">
<b>Note:</b> Amazon Bedrock users need to request access to models before they are available for use. If you want to add additional models for text, chat, and image generation, you need to request access to models in Amazon Bedrock. To request access to additional models, select the Model access link in the left side navigation panel in the Amazon Bedrock console. For more information see: <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html">https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html</a>
</div>

In this example we will be using different models of **Anthropic Claude on Amazon Bedrock**. The biggest model, Sonnet 3.5, will be in charge of planning the execution, while the smaller and faster, Haiku 3, will execute the plan. For this, you will need to request access to:

- Planning model: **Sonnet 3.5 or Opus 3**
- Execution model: **Haiku 3**

## Set up the environment

First, let's install the required libraries. Boto3 version should be > 1.34.123 in order to support the [Amazon Bedrock Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html)

In [1]:
!pip install boto3==1.34.123

Let's now import the required libraries.

In [2]:
import boto3
import json
import sqlite3
import time
import uuid
from ast import literal_eval
from botocore.exceptions import ClientError

In [3]:
print(boto3.__version__)

1.34.123


**Load variables saved in prior Notebook**

At the end of **Retail_02_Training_Layer** we saved some variables that we'll need in this Notebook. The following cell will load those variables into this lab environment. You must run at least **Retail_01_Data_Layer** and **Retail_02_Training_Layer]** before running this notebook.

In [4]:
%store -r

**Let's initialize Amazon Personalize and Amazon Bedrock clients**

In [5]:
# Initialize the Amazon Personalize service client
personalize = boto3.client('personalize')

# Initialize the Amazon Personalize Runtime service client
personalize_runtime = boto3.client(  
    service_name='personalize-runtime',
    region_name=region
    )

# Initialize the Amazon Personalize Events service client
personalize_events = boto3.client(  
    service_name='personalize-events',
    region_name=region
    )

# Initialize the Amazon Bedrock Runtime service client
client = boto3.client("bedrock-runtime", region_name=region)

Test Amazon Bedrock Converse API

**NOTE:** You need to have access to Claude Sonnet 3.5. Else, you will need to modify the MODEL_NAME_1 variable in the following cell to Opus (anthropic.claude-3-opus-20240229-v1:0) or Sonnet 3 (anthropic.claude-3-sonnet-20240229-v1:0) for the Planning model.



In [6]:
messages = [{"role": "user", "content": [{"text": "What is your name?"}]}]

MODEL_NAME_1 = "anthropic.claude-3-sonnet-20240229-v1:0"
MODEL_NAME_2 = "anthropic.claude-3-haiku-20240307-v1:0"

response = client.converse(
    modelId=MODEL_NAME_1,
    messages=messages,
)

print(response)

{'ResponseMetadata': {'RequestId': '0b970c24-719f-4c96-b176-ddcdbfbb7c8c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 22 Aug 2024 20:43:38 GMT', 'content-type': 'application/json', 'content-length': '222', 'connection': 'keep-alive', 'x-amzn-requestid': '0b970c24-719f-4c96-b176-ddcdbfbb7c8c'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': "My name is Claude. It's nice to meet you!"}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 12, 'outputTokens': 15, 'totalTokens': 27}, 'metrics': {'latencyMs': 563}}


# Create a Database with product details

We will create a local database using sqlite3 to import the retaildemostore item dataset that we used in the previous Notebook 1. We will use a yaml version of the items called **products.yaml**. This information will be used later to get product details.

In [7]:
!wget https://raw.githubusercontent.com/aws-samples/retail-demo-store/master/src/products/data/products.yaml -O ./products.yaml

--2024-08-22 20:43:38--  https://raw.githubusercontent.com/aws-samples/retail-demo-store/master/src/products/data/products.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1426674 (1.4M) [text/plain]
Saving to: ‘./products.yaml’

100%[======================================>] 1,426,674   --.-K/s   in 0.02s   

2024-08-22 20:43:38 (89.7 MB/s) - ‘./products.yaml’ saved [1426674/1426674]



In [8]:
import sqlite3
import yaml

# Read the YAML file and parse the data
with open('products.yaml', 'r') as file:
    products = yaml.safe_load(file)

# Create an SQLite database and table for storing the products
conn = sqlite3.connect('products.db')
cursor = conn.cursor()

# Create the products table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        id TEXT PRIMARY KEY,
        category TEXT,
        current_stock INTEGER,
        description TEXT,
        gender_affinity TEXT,
        image TEXT,
        image_license TEXT,
        link TEXT,
        name TEXT,
        price REAL,
        style TEXT,
        where_visible TEXT,
        promoted BOOLEAN
    )
''')

# Insert the products into the SQLite table
for product in products:
    # Prepare the product data with null values for missing fields
    product_data = (
        product.get('id', None),
        product.get('category', None),
        product.get('current_stock', None),
        product.get('description', None),
        product.get('gender_affinity', None),
        product.get('image', None),
        product.get('image_license', None),
        product.get('link', None),
        product.get('name', None),
        product.get('price', None),
        product.get('style', None),
        product.get('where_visible', None),
        product.get('promoted', False)
    )

    # Insert the product into the table
    cursor.execute('''
        INSERT OR IGNORE INTO products (
            id, category, current_stock, description, gender_affinity, image,
            image_license, link, name, price, style, where_visible, promoted
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', product_data)

# Commit the changes and close the connection
conn.commit()
conn.close()


## Define the client-side tools

Next, we'll define the client-side tools that our agent will use to assist customers. We'll create seven tools: **get_customer_id, get_customer_orders, get_order_details, purchase_order, get_recommended_products, get_similar_items** and **get_product_details.**

The agent is designed to assist customers with various e-commerce related tasks, such as retrieving customer information, placing orders, and getting product recommendations. 

These tools represent the capabilities that the agent will have access to when interacting with customers. Each tool is defined with a specific purpose and input schema, allowing the agent to use them appropriately based on the customer's needs.

Here's a brief overview of each tool:

- get_customer_id: Retrieves a customer's ID based on their name.
- get_customer_orders: Fetches a customer's order information using their ID.
- get_order_details: Provides detailed information about a specific order.
- purchase_order: Processes a product purchase for a customer.
- get_recommended_products: Suggests products to a customer based on their ID and optional category filter.
- get_similar_items: Finds similar products to a given item.
- get_product_details: Retrieves detailed information about specific products.

It's important to note that this code snippet only defines the specifications for these tools. The actual implementation of these functions will be created later in this notebook. These specifications serve as a blueprint for what each tool can do and what input it requires.

For more information on building AI agents with Amazon Bedrock using tools, you might want to refer to the Amazon Bedrock Converse API tool use documentation: https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html 

These tools will allow your agent to perform a wide range of customer service and recommendation tasks, creating a personalized shopping experience for users.

**Note:** Tool use with models is also known as **Function calling**.

In [9]:
tools = [
    {
        "toolSpec": {
            "name": "get_customer_id",
            "description": "Useful to retrieve the customer id if the customer name has been provided. Returns the customer's id",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "customer_name": {
                            "type": "string",
                            "description": "The unique name of the customer."
                        }
                    },
                    "required": ["customer_name"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "get_customer_orders",
            "description": "Retrieves customer information based on their customer ID. Returns the order ID.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "customer_id": {
                            "type": "string",
                            "description": "The unique identifier for the customer."
                        }
                    },
                    "required": ["customer_id"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "get_order_details",
            "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product id, quantity, price, and order status.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "order_id": {
                            "type": "string",
                            "description": "The unique identifier for the order."
                        }
                    },
                    "required": ["order_id"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "purchase_order",
            "description": "Purchases an product based on the provided product ID and customer id. The customer_id is always retrieved previously using the get_customer_id tool. Returns a confirmation message if the purchase is successful.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "customer_id": {
                            "type": "string",
                            "description": "The unique identifier for the customer who is purchasing the product."
                        },
                        "product_id": {
                            "type": "string",
                            "description": "The unique identifier for the product id to be purchased. Use"
                        }
                    },
                    "required": ["customer_id", "product_id"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "get_recommended_products",
            "description": "Retrieves recommended products based on the customer's customer ID and category filter. Returns the product ids.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "customer_id": {
                            "type": "string",
                            "description": "The unique identifier for the customer."
                        },
                        "category_filter": {
                            "type": "string",
                            "enum": [
                                "accessories",
                                "apparel",
                                "beauty",
                                "books",
                                "cold dispensed",
                                "electronics",
                                "floral",
                                "food service",
                                "footwear",
                                "furniture",
                                "groceries",
                                "homedecor",
                                "hot dispensed",
                                "housewares",
                                "instruments",
                                "jewelry",
                                "outdoors",
                                "salty snacks",
                                "seasonal",
                                "tools"
                            ],
                            "description": "The category filter values for the filter. If the category is not in the list, category filter is not applied. Optional."
                        }
                    },
                    "required": ["customer_id"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "get_similar_items",
            "description": "Retrieves similar items for a given product id. Returns a list of product ids for the similar items",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "product_id": {
                            "type": "string",
                            "description": "The id of the product for which similar items are to be found."
                        },
                        "customer_id": {
                            "type": "string",
                            "description": "The unique identifier for the customer who is purchasing the product."
                        }
                    },
                    "required": ["product_id", "customer_id"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "get_product_details",
            "description": "Retrieves the details of a list of products based on the product IDs provided as an array. Returns the products' details such as name, description and price",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "product_ids": {
                            "type": "array",
                            "items": {
                                "type": "string"
                            },
                            "description": "An array containing the unique identifiers for the products."
                        }
                    },
                    "required": ["product_ids"]
                }
            }
        }
    }
]

## Tool definition

We will create a mix of tools that use synthetic data and tools that will call Amazon Personalize resources created in [01_Recommender-Agent_Configure-Personalize-Resources.ipynb](./01_Recommender-Agent_Configure-Personalize-Resources.ipynb)

These functions correspond to the tools we defined earlier for our personalized recommender agent. They represent the actual implementation of the capabilities that the agent will have access to when assisting customers. It's crucial to understand that these functions are the resources the agent can execute, but the Large Language Model (LLM) itself does not directly execute these functions.

Here's how the process works:

1. The LLM (in this case, Claude) analyzes the user's request and formulates a plan using the available tools.
2. When the LLM determines that a tool should be used, it doesn't execute the function itself. Instead, it indicates which tool should be used and with what parameters.
3. The agent framework (which is separate from the LLM) then executes the corresponding function and returns the result.
4. The LLM receives the result of the tool execution and uses this information to continue its plan or formulate a response to the user.
5. This process of planning, tool use, and result interpretation continues until the LLM can generate a final answer without needing additional tool use.

In the provided code, we see a mix of tools:

- Some use synthetic data (like get_customer_id and get_order_details), which is useful for testing and development.
- Others interact with real services, such as Amazon Personalize (like **get_recommended_products, get_similar_items** and **purchase_order**), which were set up in a previous notebook.

This combination allows for a realistic simulation of a production environment while still providing controlled data for certain operations.
Remember, the LLM's role is to understand the user's request, plan the necessary steps using these tools, and interpret the results to provide a coherent response to the user. The actual execution of these functions is handled by the agent framework, creating a powerful and flexible system for personalized customer assistance.

In [10]:
def get_customer_id(customer_name):
    # Simulated customer data
    customers = {
        "John Doe": {"id": "1234", "email": "john@example.com", "phone": "123-456-7890"},
        "Jane Smith": {"id": "1235", "email": "jane@example.com", "phone": "987-654-3210"},
        "Sergio Garcés": {"id": "1236", "email": "sergio@example.com", "phone": "987-654-9999"}
    }
    return customers.get(customer_name, "Customer not found")

def get_customer_orders(customer_id):
    # Simulated order data
    orders = {
        "1234": {"id": "O1"},
        "1235": {"id": "O2"},
        "1236": {"id": "O3"},
    }
    return orders.get(customer_id, "Customer not found")

def get_order_details(order_id):
    # Simulated order data
    orders = {
        "O1": {"id": "O1", "product": "2f3f075b-66ff-475f-95e3-71b462f328c5", "quantity": 2, "price": 145.0, "status": "Shipped"},
        "O2": {"id": "O2", "product": "092a278e-4d04-4ce3-bc18-e8a0491a9fb5", "quantity": 1, "price": 23.99, "status": "Processing"},
        "O3": {"id": "O3", "product": "54dc5dd9-3df4-410a-ae55-54844988a0ca", "quantity": 1, "price": 31.99, "status": "Shipped"},
    }
    return orders.get(order_id, "Order not found")

def purchase_order(customer_id, product_id):

    # Purchase the product
    response = personalize_events.put_events(
        trackingId = tracking_id,
        userId = customer_id,
        sessionId = str(uuid.uuid4()),
        eventList = [
            {
                'eventId': str(uuid.uuid4()),
                'eventType': 'Purchase',
                'itemId': product_id,
                'sentAt': int(time.time()),
                'properties': '{"discount": "No"}'
            }
        ]
    )

    # Wait for Purchase event to become consistent.
    time.sleep(10)

    # Check if the response is correct
    if response["ResponseMetadata"]["HTTPStatusCode"] == 200:
        return "Product purchased successfully!"
    else:
        return "Error purchasing product."

def get_recommended_products(customer_id, category_filter=None):
    # Define the filter parameters
    filter_values = {}

    if category_filter or category_filter == "None":
        
        filter_values = {"CATEGORY" : "\"" + category_filter + "\""}
    # If category_filter is provided, add filter parameters to the API call
    if filter_values:
        get_recommendations_response = personalize_runtime.get_recommendations(
            recommenderArn=workshop_recommender_recommended_for_you_arn,
            userId=str(customer_id),
            numResults=3,
            filterArn=category_filter_arn,
            filterValues=filter_values
        )
    else: 
        # Call the Amazon Personalize Runtime service to get recommendations
        get_recommendations_response = personalize_runtime.get_recommendations(
            recommenderArn=workshop_recommender_recommended_for_you_arn,
            userId=str(customer_id),
            numResults=3,
        )

    # Extract the recommended item IDs and product descriptions from the response
    recommended_items = [item['itemId'] for item in get_recommendations_response['itemList']]

    return recommended_items

def get_similar_items(product_id, user_id):
    # Call the Amazon Personalize Runtime service to get similar items
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn=workshop_recommender_customers_who_viewed_arn,
        itemId=str(product_id),
        userId=str(user_id),
        numResults=2,
    )

    # Extract the similar item IDs from the response
    similar_item_ids = [item['itemId'] for item in get_recommendations_response['itemList']]

    return similar_item_ids

import ast

def get_product_details(product_ids):
    # Check if product_ids is a string
    if isinstance(product_ids, str):
        # Convert the string to a list of product IDs
        product_ids = literal_eval(product_ids)

    # Connect to the database
    conn = sqlite3.connect('products.db')
    cursor = conn.cursor()

    # Define the SQL query
    query = '''
        SELECT * FROM products
        WHERE id IN ({})
    '''.format(', '.join(['?'] * len(product_ids)))

    # Execute the query
    cursor.execute(query, product_ids)

    # Fetch the results
    results = cursor.fetchall()

    # Close the connection
    conn.close()

    # Return the product details or None if not found
    product_details = []
    for result in results:
        product_detail = {
            "id": result[0],
            "category": result[1],
            "current_stock": result[2],
            "description": result[3],
            "gender_affinity": result[4],
            "image": result[5],
            "image_license": result[6],
            "link": result[7],
            "name": result[8],
            "price": result[9],
            "style": result[10],
            "where_visible": result[11],
            "promoted": result[12]
        }
        product_details.append(product_detail)

    if product_details:
        return product_details
    else:
        return "Products not found"




    

### Test tools

Let's test the get_recommended_products tool that will retrieve recommended products for a given user.

In [11]:
get_recommended_products('1234')

['afba8eed-c114-422f-8960-13f90d2657ff',
 '274758fa-fd8a-4b25-b87f-fa6b56ef9f48',
 '30ec1f80-71e1-45f7-97d5-97150daea152']

In [12]:
get_product_details(['dff3f75f-838d-4b4d-9c7c-cee7ae2aac10'])

[{'id': 'dff3f75f-838d-4b4d-9c7c-cee7ae2aac10',
  'category': 'footwear',
  'current_stock': 13,
  'description': 'Elevate your style with these exquisite brown leather double monk strap shoes. Crafted with premium materials, these shoes feature a sleek silhouette and iconic buckle closures, perfect for both formal occasions and smart-casual ensembles. The rich cognac hue and polished finish exude sophistication and timeless elegance.',
  'gender_affinity': 'M',
  'image': 'dff3f75f-838d-4b4d-9c7c-cee7ae2aac10.jpg',
  'image_license': None,
  'link': None,
  'name': 'Classic Double Monk Strap Shoes',
  'price': 279.99,
  'style': 'formal',
  'where_visible': 'UI',
  'promoted': 0}]

Let's test the get_recommended_products tools that will retrieve recommend products for a given user

## Process tool calls and return results

We'll create a function to process the tool calls made by Claude and return the appropriate results. The `process_tool_call` function is crucial for bridging the gap between the language model's decisions and the actual execution of tools in our system. Here's why it's important and how it works:

### What is a tool call?
A tool call is a request made by the language model (Claude 3) to use a specific tool with certain input parameters. It's the model's way of indicating that it needs to use a particular function to gather information or perform an action in response to a user's query.

### Why do we need to process tool calls?
We need to process tool calls for several reasons:
1. The language model doesn't directly execute code or access data sources.
2. We need to translate the model's high-level requests into actual function calls in our system.
3. It allows us to control and monitor what actions are being taken on behalf of the model.
4. We can add error handling, logging, or additional logic as needed.

### How the model selects tools
The model selects tools based on its understanding of:
- The user's query
- The available tools and their descriptions
- The current context of the conversation
- What information it needs to answer the query or perform the requested task

The model uses its reasoning to determine which tool is most appropriate for gathering the necessary information or performing the required action.

### The process_tool_call function
This function acts as a bridge between the model's high-level tool requests and the actual function calls in our system. It:
- Takes the tool name and input parameters as arguments
- Maps these to the corresponding Python functions we defined earlier
- Calls the appropriate function with the given inputs
- Returns the result back to the model

This setup allows the language model to make decisions about what information or actions are needed, while keeping the actual execution of these actions under the control of our system. It's a way of giving the model access to external data and capabilities without giving it direct control over the system's resources.

In [13]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_customer_id":
        return get_customer_id(tool_input["customer_name"])
    elif tool_name == "get_customer_orders":
        return get_customer_orders(tool_input["customer_id"])
    elif tool_name == "get_order_details":
        return get_order_details(tool_input["order_id"])
    elif tool_name == "purchase_order":
        return purchase_order(tool_input["customer_id"], tool_input["product_id"])
    elif tool_name == "get_recommended_products":
        category_filter = tool_input.get("category_filter")
        if category_filter:
            return get_recommended_products(tool_input["customer_id"], category_filter)
        else:
            return get_recommended_products(tool_input["customer_id"])
    elif tool_name == "get_similar_items":
        return get_similar_items(tool_input["product_id"], tool_input["customer_id"])
    elif tool_name == "get_product_details":
        return get_product_details(tool_input["product_ids"])

## Interact with the chatbot

Now, we'll create a function to manage the interaction between the user and our personalized recommender agent. This function will encapsulate the core logic of our agent, handling the flow of information between the user, the language model (Claude 3), and our defined tools.

### Agent's Logic Flow

1. **Initialization**: 
   - Set up the initial context and system prompt for the agent.
   - This prompt will guide the agent's behavior and set the context for our e-commerce recommender system.

2. **User Input**: 
   - Receive the user's message or query.

3. **LLM Processing**:
   - Send the user's input, along with the current context and system prompt, to Claude.
   - Claude analyzes the input and determines the next action (either responding directly or using a tool or a set of tools).

4. **Tool Execution (if needed)**:
   - If Claude decides to use a tool, our function will:
     a. Extract the tool name and parameters from Claude's response.
     b. Call the `process_tool_call` function to execute the appropriate tool.
     c. Capture the tool's output.

5. **Result Interpretation**:
   - Send the tool's output back to Claude for interpretation.
   - Claude may decide to use another tool or formulate a final response.

6. **Response Generation**:
   - Once Claude has gathered all necessary information, it generates a final response to the user.

7. **Conversation Update**:
   - Update the conversation history with the user's input and the agent's response.

8. **Repeat**:
   - The process repeats for each user input, maintaining context throughout the conversation.

### System Prompt

The system prompt is crucial as it sets the tone, capabilities, and limitations of our agent. Our system prompt includes:

- The agent's role as an AI assistant for product purchases and order management
- A step-by-step guide for the agent to follow when assisting customers
- Rules for interaction and information gathering
- Guidelines for tool usage and response formulation

By encapsulating this logic in a single function, we create a modular and maintainable structure for our chatbot. This allows for easy updates to the agent's behavior and capabilities as needed.

Let's first create the system message variable

In [14]:
system_message = """
You are an AI assistant designed to help customers with various tasks related to product purchases and order management. 
Your goal is to understand the user's request, break it down into steps, 
and use the available tools to gather the necessary information to provide a comprehensive and accurate response.

To effectively assist the user, follow these steps:

1. Ask for the customer name or customer ID before proceeding to use any tool.

2. Carefully read and understand the customer's request or query.

3. Identify the relevant tools needed to gather the required information based on the user's request.

4. For each tool, analyze its input schema and determine the necessary inputs to call the tool successfully.

5. Execute the tools in a logical sequence, using the outputs from previous tools as inputs for the next tool when required.

6. After executing all the relevant tools, analyze the collected information and formulate a clear and concise response that addresses the user's request.

7. If additional information is needed to provide a complete response, politely ask the user for the required details or clarification.

8. If the user's request cannot be fully addressed with the available tools, provide an honest explanation and suggest alternative solutions or recommendations.

Remember, your goal is to provide the customer with accurate and helpful information while ensuring a smooth and efficient experience. Be patient, clear, and thorough in your responses.

Before providing a response for the customer check if it mets the rules provided inside the <rules> tags. Be nice and always respond as if you were speaking with the customer directly.

<rules>
- Ask for the customer name or customer ID if it has not been provided and a tool requieres the customer ID as an input,
- Don't make up the customer name if it has not been provided. For example: John Smith
- Always use get_product_details tool if you need to provide information related to a product to a user. Provide a readable response instead of IDs.
- When using get_product_details tool, remember that the input parameter is an array. Do not wrap the array with quotes (") since it will change the array to a string.
- Add all the necessary details to the final response in order for the customer to understand what you did on his behalf, but do not disclose your thinking steps inside the <thinking> tags. 
- Always double check if you have all the information you need by analysing the whole content and the thinking steps provided inside the <thinking> tags provided at the beginning. 
- Do not infer names for products. If necessary, before providing the final answer call the get_product_details tool to provide a meaninful answer to the user.
- Speak in first person to the user.
</rules>
"""


Now, let's create the main function chatbot interaction. This function will accept as arguments the user message and the chat history if available. Including the chat history is very important in order for the model to have the entire context before deciding the next step.

## Chatbot Interaction Function

This function encapsulates the core logic of our chatbot interaction:

1. **Initialization**: It sets up the conversation with the user's message and any existing chat history by extending the messages list.

2. **Initial Planning**: It makes a call to a larger language model (`MODEL_NAME_1`) for comprehensive response planning.

3. **Tool Use Loop**: If tools are required, it enters a loop where it:
   a. Extracts tool use information
   b. Processes the tool call
   c. Prepares the result for the next model call
   d. Calls a smaller, quicker model (`MODEL_NAME_2`) for subsequent interactions

4. **Iteration**: This loop continues until no more tool use is required.

5. **Response Generation**: Finally, it extracts the final response text and returns it along with the updated message history.

### Optimization Strategy

The use of two different models (a larger one for initial planning and a smaller one for subsequent interactions) is an optimization technique. This approach aims to balance between:

- Comprehensive planning
- Quick responses
- Cost reduction
- Improved response times

By using a larger model for initial planning and a smaller, faster model for follow-up actions, we can potentially achieve better performance and efficiency in our chatbot interactions.

In [15]:
import json
import time

def chatbot_interaction(user_message, chat_history=None):
    # Print a separator and the user's message for clarity
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    # Initialize the messages list with chat history if available
    messages = []
    if chat_history:
        messages.extend(chat_history)  # Extend the messages list with chat_history
    messages.append({"role": "user", "content": [{"text": user_message}]})
    
    time.sleep(1)

    # Initial call to the larger model (MODEL_NAME_1) for planning
    response = client.converse(
        modelId=MODEL_NAME_1,
        inferenceConfig={
            'maxTokens': 4096,
            'temperature': 0,
        },
        messages=messages,
        system=[
            {
                 'text': system_message
            },
        ],
        toolConfig={"tools": tools}
    )

    # Print the initial response for debugging
    print(f"\nInitial Response:")
    print(f"Stop Reason: {response['stopReason']}")
    print(f"Content: {response['output']['message']['content']}")

    # Loop to handle tool calls
    while response['stopReason'] == "tool_use":
        time.sleep(0.5)
        print("using tool")
        # Extract tool use information
        tool_use = next(block for block in response['output']['message']['content'] if isinstance(block, dict) and 'toolUse' in block)
        tool_name = tool_use['toolUse']['name']
        tool_input = tool_use['toolUse']['input']
        tool_use_id = tool_use['toolUse']['toolUseId']

        # Print tool use information for debugging
        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))

        # Process the tool call and get the result
        tool_result = process_tool_call(tool_name, tool_input)

        # Print tool result for debugging
        print(f"\nTool Result:")
        print(json.dumps(tool_result, indent=2))

        # Prepare messages for the next model call including the tool execution results
        messages_temp = [
            {"role": "assistant", "content": response['output']['message']['content']},
            {
                "role": "user",
                "content": [
                    {
                        "toolResult": {
                            "toolUseId": tool_use_id, 
                            "content": [
                                {
                                    "json": {
                                        "result": tool_result
                                    }
                                }
                            ]                        
                        }
                    }
                ],
            },
        ]
        
        messages.extend(messages_temp)
        
        # Call the smaller model (MODEL_NAME_2) for subsequent interactions
        response = client.converse(
            modelId=MODEL_NAME_2,
            inferenceConfig={
                'maxTokens': 4096,
                'temperature': 0,
            },
            messages=messages,
            system=[
                {
                    'text': system_message
                },
            ],
            toolConfig={"tools": tools}
        )

        # Print the response for debugging
        print(f"\nResponse:")
        print(f"Stop Reason: {response['stopReason']}")
        print(f"Content: {response['output']['message']['content']}")

    # Extract the final response text
    final_response = next(
        (block['text'] for block in response['output']['message']['content'] if 'text' in block),
        None,
    )

    # If there are no objects with the "text" attribute, set final_response to None
    if not final_response:
        final_response = None

    print(f"\nFinal Response: {final_response}")

    # Add the final response to the messages list
    messages.append({"role": "assistant", "content": [{"text": final_response}]})


    return final_response, messages

For better understanding, see the flowchart for function chatbot_interaction:
    
<div style="text-align: center;">
    <img src="static/function-flowchart.png" alt="Flowchart of the function" width="50%">
</div>

## Test the chatbot
Let's test our recommender agent with a few sample queries.

In [16]:
# For the first iteration
user_message = "Hello!"
final_response, chat_history = chatbot_interaction(user_message)


User Message: Hello!

Initial Response:
Stop Reason: end_turn
Content: [{'text': 'Hello! How can I assist you today? To better understand your needs, could you please provide your customer name or customer ID?'}]

Final Response: Hello! How can I assist you today? To better understand your needs, could you please provide your customer name or customer ID?


In [17]:
# For subsequent iterations
user_message = "My name is John Doe"
final_response, chat_history = chatbot_interaction(user_message, chat_history)


User Message: My name is John Doe

Initial Response:
Stop Reason: tool_use
Content: [{'text': 'Okay, got it. Let me retrieve your customer ID using the provided name.'}, {'toolUse': {'toolUseId': 'tooluse_1ea3bAzhR-uwdUdtJRd6iA', 'name': 'get_customer_id', 'input': {'customer_name': 'John Doe'}}}]
using tool

Tool Used: get_customer_id
Tool Input:
{
  "customer_name": "John Doe"
}

Tool Result:
{
  "id": "1234",
  "email": "john@example.com",
  "phone": "123-456-7890"
}

Response:
Stop Reason: end_turn
Content: [{'text': "Great, I've retrieved your customer ID as 1234. How can I help you today?"}]

Final Response: Great, I've retrieved your customer ID as 1234. How can I help you today?


In [18]:
# For the first iteration
user_message = "Show me the product name John Doe has ordered"
final_response, chat_history = chatbot_interaction(user_message)


User Message: Show me the product name John Doe has ordered

Initial Response:
Stop Reason: tool_use
Content: [{'text': 'To show you the product name that John Doe has ordered, I will need to follow these steps:\n\n<thinking>\n1. Get the customer ID for "John Doe" using the get_customer_id tool\n2. Use the customer ID to retrieve John Doe\'s orders with the get_customer_orders tool\n3. For each order ID, use the get_order_details tool to get the product ID\n4. Use the get_product_details tool with the product IDs to retrieve the actual product names\n</thinking>'}, {'toolUse': {'toolUseId': 'tooluse_JWcuC6F7QgSGgDLjxqoAUw', 'name': 'get_customer_id', 'input': {'customer_name': 'John Doe'}}}]
using tool

Tool Used: get_customer_id
Tool Input:
{
  "customer_name": "John Doe"
}

Tool Result:
{
  "id": "1234",
  "email": "john@example.com",
  "phone": "123-456-7890"
}

Response:
Stop Reason: tool_use
Content: [{'text': 'Okay, I was able to retrieve the customer ID for John Doe, which is "

In [19]:
# For subsequent iterations
user_message = "What is the description for the product?"
final_response, chat_history = chatbot_interaction(user_message, chat_history)


User Message: What is the description for the product?

Initial Response:
Stop Reason: end_turn
Content: [{'text': 'Here is the description for the "Earthly Hues Eyeshadow Quad" product that John Doe ordered:\n\n"Indulge in the rich, earthy tones of our Earthly Hues Eyeshadow Quad. Each palette features four harmonious shades ranging from sultry purples to warm neutrals, perfect for creating versatile looks from day to night. The finely-milled, highly-pigmented formula glides on smoothly for effortless blending and long-lasting wear."\n\nThis eyeshadow quad contains four complementary earthy and neutral shades in a highly pigmented, smooth formula that allows for easy blending and all-day wear. The shades range from sultry purples to warm neutrals to create versatile makeup looks.'}]

Final Response: Here is the description for the "Earthly Hues Eyeshadow Quad" product that John Doe ordered:

"Indulge in the rich, earthy tones of our Earthly Hues Eyeshadow Quad. Each palette features 

In [20]:
# Single user request with multiple tasks
user_message = "Hello, my name is John Doe. What are the top electronics products recommended for me and which one is cheaper?"
final_response, chat_history = chatbot_interaction(user_message)


User Message: Hello, my name is John Doe. What are the top electronics products recommended for me and which one is cheaper?

Initial Response:
Stop Reason: tool_use
Content: [{'text': 'Hello John Doe, to provide you with recommended electronics products and their prices, I will need to follow these steps:\n\n<thinking>\n1. Get the customer ID for "John Doe" using the get_customer_id tool\n2. Use the get_recommended_products tool with the customer ID to get recommended product IDs in the electronics category\n3. Use the get_product_details tool with the recommended product IDs to get the product names, descriptions and prices\n4. Compare the prices of the recommended electronics products and identify the cheaper one\n</thinking>'}, {'toolUse': {'toolUseId': 'tooluse_i0DdkHcpT0Oecm7uETEW1w', 'name': 'get_customer_id', 'input': {'customer_name': 'John Doe'}}}]
using tool

Tool Used: get_customer_id
Tool Input:
{
  "customer_name": "John Doe"
}

Tool Result:
{
  "id": "1234",
  "email": 

In [21]:
# Single user request with multiple tasks
user_message = "Hello, my name is John Doe, what is the top recommended product? buy the product"
final_response, chat_history = chatbot_interaction(user_message)


User Message: Hello, my name is John Doe, what is the top recommended product? buy the product

Initial Response:
Stop Reason: tool_use
Content: [{'text': 'Hello John Doe, to provide you with the top recommended product, I will first need to retrieve your customer ID. Let me do that now:'}, {'toolUse': {'toolUseId': 'tooluse_yyZv-_bASbawzXBns6lOXQ', 'name': 'get_customer_id', 'input': {'customer_name': 'John Doe'}}}]
using tool

Tool Used: get_customer_id
Tool Input:
{
  "customer_name": "John Doe"
}

Tool Result:
{
  "id": "1234",
  "email": "john@example.com",
  "phone": "123-456-7890"
}

Response:
Stop Reason: tool_use
Content: [{'text': 'Great, I was able to retrieve your customer ID as "1234". Now I can use that to get the top recommended products for you:'}, {'toolUse': {'toolUseId': 'tooluse_LwnbyF6uQvuv1FxIh3kBAg', 'name': 'get_recommended_products', 'input': {'customer_id': '1234'}}}]
using tool

Tool Used: get_recommended_products
Tool Input:
{
  "customer_id": "1234"
}

Too

In [22]:
# Single user request with multiple tasks. Compare prices
user_message = "Hello I am John Doe. What is the top electronic product recommended for me and similar products? Compare prices and buy the cheapest one"
final_response, chat_history = chatbot_interaction(user_message)


User Message: Hello I am John Doe. What is the top electronic product recommended for me and similar products? Compare prices and buy the cheapest one

Initial Response:
Stop Reason: tool_use
Content: [{'text': 'Hello John Doe, to provide you with the top recommended electronic product and similar items along with their prices, I will need to follow these steps:\n\n<thinking>\n1. Get customer ID for "John Doe" using the get_customer_id tool\n2. Use the customer ID to get recommended products in the "electronics" category using get_recommended_products tool\n3. From the recommended products, identify the top electronic product\n4. Use get_similar_items tool to find similar products for the top recommended electronic product\n5. Get product details like name, description and price for the top recommended product and similar items using get_product_details tool\n6. Compare prices and identify the cheapest product\n7. Use purchase_order tool to purchase the cheapest product\n</thinking>'}

In [23]:
# Single user request with multiple tasks. Purchase product
user_message = "Hello I am John Doe. What is the top product recommended for me and its similar items? Compare prices and buy the cheapest one"
final_response, chat_history = chatbot_interaction(user_message)


User Message: Hello I am John Doe. What is the top product recommended for me and its similar items? Compare prices and buy the cheapest one

Initial Response:
Stop Reason: tool_use
Content: [{'text': "Hello John Doe, to get your recommended products and similar items, I'll need to follow these steps:\n\n<thinking>\n1. Get John Doe's customer ID using the get_customer_id tool with his name as input\n2. Use the get_recommended_products tool with John's customer ID to get his top recommended product\n3. Use the get_similar_items tool with John's customer ID and the top recommended product ID to get similar items\n4. Use the get_product_details tool to get the name, description and price for the recommended product and similar items\n5. Compare prices and identify the cheapest option\n6. Use the purchase_order tool with John's customer ID and the cheapest product ID to place the order\n</thinking>"}, {'toolUse': {'toolUseId': 'tooluse_m0DuCtW8QM6HCac5zgkwiA', 'name': 'get_customer_id', '

## Build a chatbot widget for the agent to play around

Install the required libraries

In [24]:
!pip install ipywidgets

Now let's create the chatbot widget to play around. 

Note that in order to reset the chat history you need to execute the cell again

In [25]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define a function to handle user input
def handle_user_input(user_message):
    
    global chat_history
    # Call the chatbot_interaction function
    response, messages = chatbot_interaction(user_message, chat_history)
    
    # Update the chat history
    chat_history = messages
    
    # Print the final response in the output widget
    with chat_output:
        clear_output()
        print(f"Response: {response}")
    
    # Clear the input field
    user_input.value = ''

# Define a function to handle the button click
def handle_button_click(sender):
    handle_user_input(user_input.value)

# Create the widgets
user_input = widgets.Text(
    placeholder='Type your message here...',
    description='User:',
    disabled=False,
    continuous_update=False,  # Disable continuous update
    on_submit=handle_user_input  # Call handle_user_input on Enter key press
)

chat_output = widgets.Output()

# Initialize the chat history
chat_history = []

# Create a button and link the handle_button_click function
send_button = widgets.Button(description='Send')
send_button.on_click(handle_button_click)

# Display the widgets
display(widgets.HBox([user_input, send_button]))
print("\n")
display(chat_output)
print("\n")

Output()

Try having a conversation as if you would in real life. For example:

- Hello, my name is John Doe, I would like to know the status of my order?
- What is the product name?
- What do other people buy together with that product?
- What is the price of those?
- Ok, purchase both

If you are done with this sample, make sure to follow the steps in the next section to cleanup the resources created in this notebook.

## Cleanup Resources 
The next notebook contains instructions on how to clean up the resources created in this notebook and the previous notebooks

### Run the cleanup notebook

Continue to [Retail_06_Clean_Up.ipynb](./03_Recommender-Agent_CleanUp.ipynb)
clean up resources.